# Credit Risk Modeling | Part 4: Checking Populating Stability Index (PSI)

<img src = "https://insightconsultants.co/wp-content/uploads/2020/05/creditscore.jpg" width = '750'>


We obtained the credit risk models with the loan data from 2007 to 2014. In this notebook, we will check if the model performs well with the loan data in 2015.  
  
Credit score is the outcome that our probability model gives us, or, in other words, the assessment of credit worthiness. The substantial difference between the two populations shows that it is necessary to build a new PD model which corresponds more closely to the most recent population.

# 1 Data Preparation

## Importing Libraries

In [ ]:
#installing gdown package to download dataset stored in G Drive
!pip install gdown
# to upgrade
!pip install --upgrade gdown

In [2]:
#libs for managing data
import numpy as np
import pandas as pd

#libs for ML regression model
from sklearn.linear_model import LogisticRegression
from sklearn import metrics  
from sklearn.metrics import roc_curve, roc_auc_score

#libs for plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#exporting ML model
import pickle

#lib to download dataset on g drive
import gdown

#setting display option max rows
#pd.options.display.max_rows = None

#importing custom-made functions
import sys #importing local functions in src folder
sys.path.append('../src/')
from functions import *

## Importing Data

In [3]:
# #loading dataset from Gdrive
# #fname_2007etp = "https://drive.google.com/file/d/16JXrTBSgEJH4_30zlFFBye1GRHh5h4O0/view?usp=share_link"
# #fname_2007etp = 'https://drive.google.com/uc?id=' + fname_2007etp.split('/')[-2]

# #updated dataset
# fname_2015 = "https://drive.google.com/file/d/1Fb7LFd97aJm0ySb0A48_znfe9KQzaUZO/view?usp=share_link"
# fname_2015 = 'https://drive.google.com/uc?id=' + fname_2015.split('/')[-2]

In [ ]:
# # downloading gdrive files
# url = fname_2015
# output = "loan_data_2015.csv"
# gdown.download(url, output, quiet=False)

In [ ]:
loan_data= pd.read_csv(output)

In [6]:
# test and train datasets were preprocessed in the first notebook.
# they were than stocked over G-drive for the easiness. 
# the links below download the training dataset  from G-Drive
# url = "https://drive.google.com/file/d/1v45P0HhUxZ1x4HmD0bWblpUyUWuiC0_5/view?usp=share_link"
# output = "loan_data_inputs_train.csv"
# g_down (url,'../data/'+output)

In [ ]:
#loading csv files as pd dataframes
inputs_train = pd.read_csv ('../data/'+'loan_data_inputs_train.csv',index_col =0)

## Exploring Data

In [ ]:
# Uncomment below to set the pandas dataframe options to display all columns/ rows.
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None

loan_data.shape

In [ ]:
loan_data.head()

In [ ]:
loan_data.tail()

In [ ]:
loan_data.columns.values

In [ ]:
# Displaying column names with non missing cases and datatype
loan_data.info()

# 2 General Preprocessing

## a Discrete variables - Dummy columns

In [13]:
dummy_columns = ['grade','sub_grade','home_ownership','verification_status',
                 'loan_status','purpose','addr_state','initial_list_status']

df_Dummies = pd.DataFrame()
for col in dummy_columns:
  df_Dummy= pd.get_dummies(loan_data[col], prefix = col, prefix_sep = ':')
  df_Dummies = pd.concat([df_Dummies,df_Dummy ], axis=1)
  #print(loan_data_Dummies.head())
  # = pd.concat([loan_data,loan_data_Dummies],axis = 1)

In [ ]:
#list of all dummy columns. 
df_Dummies.columns.values

In [15]:
#merging dummy columns with the main dataset
loan_data = pd.concat([loan_data,df_Dummies],axis = 1)

## b Continuous variables - Dt format conversion

In [ ]:
#lets convert emp_length into integer
loan_data['emp_length'].unique()

In [19]:
# Clean and convert 'emp_length' to numeric
loan_data['emp_length_int'] = (
    loan_data['emp_length']
    .str.replace(r'\+ years?|< 1 year|n/a', '0', regex=True)  # Handle '+ years', '< 1 year', and 'n/a'
    .str.replace(r' years?| year', '', regex=True)            # Remove 'year' or 'years'
)

# Convert to numeric type
loan_data['emp_length_int'] = pd.to_numeric(loan_data['emp_length_int'], errors='coerce')

In [ ]:
# date variables not in dt format
loan_data['earliest_cr_line']

In [21]:
#converting date column to format %b-%y : Apr-03 => 2003-04-03
loan_data['earliest_cr_line_date']=pd.to_datetime(loan_data['earliest_cr_line'],format = '%b-%y')
#calculating the months since a default date taken as 2017-12-01
diff_cr_line = pd.to_datetime('2017-12-01') - loan_data['earliest_cr_line_date']
loan_data['mths_since_earliest_cr_line'] = round(pd.to_numeric( diff_cr_line / np.timedelta64(1, 'M')))

In [ ]:
loan_data['mths_since_earliest_cr_line'].describe()

In [ ]:
#finding out why there are negative values in our dataset
m1 = loan_data['mths_since_earliest_cr_line']<0
loan_data.loc[m1,['earliest_cr_line','earliest_cr_line_date','mths_since_earliest_cr_line']].head()

In [24]:
# it is neccessary to handle negative values. they are due to the 196x data read as 206x
# we take the maximum month difference to replace negative values
loan_data.loc[m1,'mths_since_earliest_cr_line'] = loan_data.loc[:,'mths_since_earliest_cr_line'].max()

In [ ]:
#let's convert term column to integer format
loan_data['term']

In [ ]:
loan_data['term_int'] = loan_data['term'].str.replace(' months', '').astype(int)
loan_data['term_int'].describe()

In [ ]:
# Assuming we are in December 2017
loan_data['issue_d_date'] = pd.to_datetime(loan_data['issue_d'], format = '%b-%y')

#calculating the month difference from 2017-12-01
# We calculate the difference between two dates in months, turn it to numeric datatype and round it.
diff_issue_d = pd.to_datetime('2017-12-01') - loan_data['issue_d_date']
loan_data['mths_since_issue_d'] = round(pd.to_numeric( diff_issue_d / np.timedelta64(1, 'M')))

# Showing some descriptive statisics for the values of a column.
loan_data['mths_since_issue_d'].describe()

## c Checking for missing values or cleaning them

In [ ]:
pd.options.display.max_rows = None
loan_data.isnull().sum().sort_values(ascending=False)

In [29]:
# filling up the empty rows that will be used in our model.
# we use funded_amnt for the missing total_rev_hi_lim values
# fundedAmnt The total amount committed to that loan at that point in time.
loan_data['total_rev_hi_lim'].fillna(loan_data['funded_amnt'],inplace = True)

In [30]:
# for the missing values in annual_inc, mean value is considered.  
loan_data['annual_inc'].fillna(loan_data['annual_inc'].mean(),inplace = True)

In [31]:
# for the missing values below, we consider 0
loan_data['mths_since_earliest_cr_line'].fillna(0,inplace = True)
loan_data['acc_now_delinq'].fillna(0, inplace=True)
loan_data['total_acc'].fillna(0, inplace=True)
loan_data['pub_rec'].fillna(0, inplace=True)
loan_data['open_acc'].fillna(0, inplace=True)
loan_data['inq_last_6mths'].fillna(0, inplace=True)
loan_data['delinq_2yrs'].fillna(0, inplace=True)
loan_data['emp_length_int'].fillna(0, inplace=True)

##**d Preprocessing of discrete variables**

We apply the same preprocessing as done in L01 notebook.


In [ ]:
#running preproc on test dataset
loan_data_2015= preproc_input_data2015 (loan_data)
loan_data_2015.columns.values

In [33]:
# We commented on some features, the coefficients for all or almost all of the dummy variables for which,
# are not statistically significant.
# Refined variables
features_all = ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
#'mths_since_earliest_cr_line:<140',
#'mths_since_earliest_cr_line:141-164',
#'mths_since_earliest_cr_line:165-247',
#'mths_since_earliest_cr_line:248-270',
#'mths_since_earliest_cr_line:271-352',
#'mths_since_earliest_cr_line:>352',
#'delinq_2yrs:0',
#'delinq_2yrs:1-3',
#'delinq_2yrs:>=4',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
#'open_acc:0',
#'open_acc:1-3',
#'open_acc:4-12',
#'open_acc:13-17',
#'open_acc:18-22',
#'open_acc:23-25',
#'open_acc:26-30',
#'open_acc:>=31',
'pub_rec:0-2',
'pub_rec:3-4',
'pub_rec:>=5',
#'total_acc:<=27',
#'total_acc:28-51',
#'total_acc:>=52',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'total_rev_hi_lim:<=5K',
'total_rev_hi_lim:5K-10K',
'total_rev_hi_lim:10K-20K',
'total_rev_hi_lim:20K-30K',
'total_rev_hi_lim:30K-40K',
'total_rev_hi_lim:40K-55K',
'total_rev_hi_lim:55K-95K',
'total_rev_hi_lim:>95K',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
#'mths_since_last_delinq:Missing',
#'mths_since_last_delinq:0-3',
#'mths_since_last_delinq:4-30',
#'mths_since_last_delinq:31-56',
#'mths_since_last_delinq:>=57',
#'mths_since_last_record:Missing',
#'mths_since_last_record:0-2',
#'mths_since_last_record:3-20',
#'mths_since_last_record:21-31',
#'mths_since_last_record:32-80',
#'mths_since_last_record:81-86',
#'mths_since_last_record:>=86',
]


In [34]:

ref_categories = ['grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'int_rate:>20.281',
#'mths_since_earliest_cr_line:<140',
#'delinq_2yrs:>=4',
'inq_last_6mths:>6',
#'open_acc:0',
'pub_rec:0-2',
#'total_acc:<=27',
'acc_now_delinq:0',
'total_rev_hi_lim:<=5K',
'annual_inc:<20K',
'dti:>35'
#'mths_since_last_delinq:0-3',
#'mths_since_last_record:0-2'
]

In [35]:
inputs_2015_With_ref_cat = loan_data_2015.loc[:,features_all]

In [ ]:
# loading values of df in a new dataframe.   
inputs_train_With_ref_categories = inputs_train.loc[: ,features_all]
 
inputs_train_With_ref_categories.head()

In [ ]:
inputs_2015_With_ref_cat.shape

# 3 Calculating scores from a score card

In [38]:
#url = 'https://drive.google.com/file/d/1XU0CwMv2skhc_P_LHiDZzUx1N0kEW2Rg/view?usp=share_link'
#output = 'df_scorecard.csv'
#g_down (url,output)

df_scorecard = pd.read_csv ('../data/df_scorecard.csv',index_col= 0)

In [ ]:
df_scorecard

In [ ]:
inputs_train_With_ref_cat_w_intercept = inputs_train_With_ref_categories
inputs_train_With_ref_cat_w_intercept.insert(0,'Intercept', 1)
inputs_train_With_ref_cat_w_intercept = inputs_train_With_ref_cat_w_intercept[df_scorecard['Feature name'].values]
inputs_train_With_ref_cat_w_intercept.head()

In [ ]:
inputs_2015_With_ref_cat_w_intercept = inputs_2015_With_ref_cat
inputs_2015_With_ref_cat_w_intercept.insert(0,'Intercept', 1)
inputs_2015_With_ref_cat_w_intercept = inputs_2015_With_ref_cat_w_intercept[df_scorecard['Feature name'].values]
inputs_2015_With_ref_cat_w_intercept.head()

In [42]:
scorecard_scores = df_scorecard['Score - Final']
scorecard_scores = scorecard_scores.values.reshape(87,1)

In [ ]:
y_scores_train = inputs_train_With_ref_cat_w_intercept.dot(scorecard_scores)
y_scores_train.head()

In [ ]:
y_scores_2015 = inputs_2015_With_ref_cat_w_intercept.dot(scorecard_scores)
y_scores_2015.head()

In [45]:
inputs_train_with_ref_cat_w_intercept = pd.concat([inputs_train_With_ref_cat_w_intercept, y_scores_train], axis = 1)
inputs_2015_with_ref_cat_w_intercept = pd.concat([inputs_2015_With_ref_cat_w_intercept, y_scores_2015], axis = 1)

In [46]:
inputs_train_with_ref_cat_w_intercept.columns.values[inputs_train_with_ref_cat_w_intercept.shape[1]-1] = 'Score'
inputs_2015_with_ref_cat_w_intercept.columns.values[inputs_2015_with_ref_cat_w_intercept.shape[1]-1] = 'Score'

In [47]:
# Creating dummy variables for score intervals with old (\"expected\")."
inputs_train_with_ref_cat_w_intercept['Score:300-350'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 300) & (inputs_train_with_ref_cat_w_intercept['Score'] < 350), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:350-400'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 350) & (inputs_train_with_ref_cat_w_intercept['Score'] < 400), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:400-450'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 400) & (inputs_train_with_ref_cat_w_intercept['Score'] < 450), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:450-500'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 450) & (inputs_train_with_ref_cat_w_intercept['Score'] < 500), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:500-550'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 500) & (inputs_train_with_ref_cat_w_intercept['Score'] < 550), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:550-600'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 550) & (inputs_train_with_ref_cat_w_intercept['Score'] < 600), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:600-650'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 600) & (inputs_train_with_ref_cat_w_intercept['Score'] < 650), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:650-700'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 650) & (inputs_train_with_ref_cat_w_intercept['Score'] < 700), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:700-750'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 700) & (inputs_train_with_ref_cat_w_intercept['Score'] < 750), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:750-800'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 750) & (inputs_train_with_ref_cat_w_intercept['Score'] < 800), 1, 0)
inputs_train_with_ref_cat_w_intercept['Score:800-850'] = np.where((inputs_train_with_ref_cat_w_intercept['Score'] >= 800) & (inputs_train_with_ref_cat_w_intercept['Score'] <= 850), 1, 0)


In [48]:
# Creating dummy variables for score intervals with new (\"actual\").
inputs_2015_with_ref_cat_w_intercept['Score:300-350'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 300) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 350), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:350-400'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 350) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 400), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:400-450'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 400) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 450), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:450-500'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 450) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 500), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:500-550'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 500) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 550), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:550-600'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 550) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 600), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:600-650'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 600) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 650), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:650-700'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 650) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 700), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:700-750'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 700) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 750), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:750-800'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 750) & (inputs_2015_with_ref_cat_w_intercept['Score'] < 800), 1, 0)
inputs_2015_with_ref_cat_w_intercept['Score:800-850'] = np.where((inputs_2015_with_ref_cat_w_intercept['Score'] >= 800) & (inputs_2015_with_ref_cat_w_intercept['Score'] <= 850), 1, 0)

# 4 Population Stability Index

It is calculated with the following formula:  
PSI = [Proportions w/ new data - Proportions w train data ] x ln (Proportions w/ new data / Proportions w train data)

In [49]:
# Creating a df with proportions for the old (\"expected\") data.
PSI_calc_train = inputs_train_with_ref_cat_w_intercept.sum() / inputs_train_with_ref_cat_w_intercept.shape[0]
# Creating a df with proportions for the new (\"actual\") data.
PSI_calc_2015 = inputs_2015_with_ref_cat_w_intercept.sum() / inputs_2015_with_ref_cat_w_intercept.shape[0]

In [50]:
# Concating two dfs along the columns.
PSI_calc = pd.concat([PSI_calc_train, PSI_calc_2015], axis = 1)

In [51]:
# Resetting the index.
PSI_calc = PSI_calc.reset_index()
# Creating a new column
PSI_calc['Original feature name'] = PSI_calc['index'].str.split(':').str[0]
# Changing col names of df
PSI_calc.columns = ['index', 'Prop_Train', 'Prop_New', 'Orig feature']
# Keeping neccessary cols
PSI_calc = PSI_calc[np.array(['index', 'Orig feature', 'Prop_Train', 'Prop_New'])]
# Removing some rows
PSI_calc = PSI_calc[(PSI_calc['index'] != 'Intercept') & (PSI_calc['index'] != 'Score')]

In [ ]:

print(PSI_calc.dtypes)


In [57]:
# Convert Prop_New to numeric, forcing errors to NaN (useful if there are non-numeric values)
PSI_calc['Prop_New'] = pd.to_numeric(PSI_calc['Prop_New'], errors='coerce')

# Now perform the PSI calculation
epsilon = 1e-10

PSI_calc['Contribution'] = np.where(
    (PSI_calc['Prop_Train'] == 0) | (PSI_calc['Prop_New'] == 0), 
    0, 
    (PSI_calc['Prop_New'] - PSI_calc['Prop_Train']) * np.log(
        (PSI_calc['Prop_New'] + epsilon) / (PSI_calc['Prop_Train'] + epsilon)
    )
)


In [ ]:
# Calculating contribution of each dummy var to PSI
# If props are 0, the contribution is 0,
# Otherwise, PSI formula is applied
PSI_calc['Contribution'] = np.where((PSI_calc['Prop_Train'] == 0) |\
                                    (PSI_calc['Prop_New'] == 0), 0,\
                                    (PSI_calc['Prop_New'] - PSI_calc['Prop_Train']) * np.log(PSI_calc['Prop_New'] / PSI_calc['Prop_Train']))


In [ ]:
PSI_calc

In [ ]:
# Summing all contributions by original feature name
PSI_calc.groupby('Orig feature')['Contribution'].sum()

Population Stability Index remains less than 0.25 for most of the categories. 
It is very high for the initial list status. It would be due to the changement of the population in the new data and the banks' policy about its definition.

This project ends up here but the models (PD, LGD, EAD) can be re-trained as shown in notebooks L02 and L03 with more recent data. 

--- End of Notebook ---
# END